In [ ]:
import polars as pl
import pandas as pd
import os
import numpy as np
import glob
from artifacts_fth_dns_fwd.config import *

In [2]:
data_dir = PROCESSED_DATA_DIR
dnsscan_data = f'{data_dir}/dnsscan'
mikrotik_measurement = f'{data_dir}/mikrotik-testing'

rate_pub_resolver_multi_pop_files = f'{data_dir}/ratelimits-multiple-pops'

scan_overview_file = f'{dnsscan_data}/scan_overview.csv.gz'
scan_df_file = f'{dnsscan_data}/udp_dataframe_complete_2025-01-06.csv.gz'
scan_df_file_aug = f'{dnsscan_data}/udp_dataframe_complete_2024-08-26.csv.gz'

freq_over_time_df_file = f'{dnsscan_data}/frequency_per_type_over_time.csv'

shapefile = f'{data_dir}/shapefiles/ne_110m_admin_0_countries.shp'

In [3]:
overview_df = pl.read_csv(scan_overview_file,separator=";")
scan_df = pl.read_csv(scan_df_file, separator=";")
scan_df_aug = pl.read_csv(scan_df_file_aug, separator=";")
frequency_odns_over_time = pl.read_csv(freq_over_time_df_file,separator=';')

In [ ]:
print("#######################")
print("######  TABLE 1  ######")
print("#######################")
#print(scan_df
(scan_df
    .filter(pl.col('response_type') == 'Transparent Forwarder')
    .select(['ip_response', 'org_response'])
    .with_columns([
        pl.when(pl.col('org_response').str.contains('GOOGLE'))
          .then(pl.lit('Google'))
        .when(pl.col('org_response').str.contains('CLOUDFLARENET'))
          .then(pl.lit('Cloudflare'))
        .when(pl.col('org_response').str.contains('OPENDNS'))
          .then(pl.lit('OpenDNS'))
        .when(pl.col('org_response').str.contains('MEGHBELA'))
          .then(pl.lit('MEGHBELA'))
        .when(pl.col('ip_response') == '178.233.140.109')
          .then(pl.lit('Turksat'))
        .when(pl.col('ip_response') == '103.88.88.88')
          .then(pl.lit('DNS Bersama'))
        .when(pl.col('ip_response') == '83.220.169.155')
          .then(pl.lit('Comss.one DNS'))
        .alias('Provider')
    ])
    .group_by(['ip_response', 'Provider'])
    .agg(pl.len().alias('Tfwd. [#]'))
    .with_columns(
        (pl.col('Tfwd. [#]') * 100 / pl.col('Tfwd. [#]').sum()).round(2).alias('Tfwd. [%]'))
    .sort('Tfwd. [#]', descending=True)
    .rename({"ip_response": "IP Address"})
    .limit(10)
    .to_pandas()
)
    #.to_latex(float_format="%.2f",index=False))

In [9]:
fingerprinting_df = pl.read_csv(f"{data_dir}/fingerprinting/fingerprinting_results.csv", separator=";")

In [10]:
fingerprinting_df = fingerprinting_df.filter(pl.col("router vendor").is_not_null())

In [11]:
fingerprinting_df = fingerprinting_df.with_columns(
    pl.when((pl.col('model version').str.starts_with('CCR')) & (pl.col('router vendor')=='Mikrotik')).then(pl.lit('Core'))
    .when(pl.col('router vendor').is_in(['Hikvision','UNV'])).then(pl.lit('NVR')).otherwise(pl.lit("CPE")).alias('Type')
)

In [12]:
fingerprinting_df = fingerprinting_df.group_by(['router vendor','Type']).agg(pl.len().alias('Devices [#]')).sort(by='Devices [#]',descending=True)

In [13]:
corder={'Mikrotik':1,
 'TP-Link':2,
 'Ubiquiti':3,
 'Fortinet':4,
 'ZTE':5,
 'Cisco':6,
 'Zyxel':7,
 'Huawei':8,
 'D-Link':9,
 'Other':10,
 'Hikvision':11,
 'UNV':12}
fingerprinting_df = fingerprinting_df.with_columns(
    pl.when(pl.col('Devices [#]')>=24).then(pl.col('router vendor')).otherwise(pl.lit('Other'))
).group_by(['router vendor','Type']).agg(pl.col('Devices [#]').sum()).sort('Devices [#]',descending=True).to_pandas().sort_values(by='router vendor',key = lambda vendor: vendor.map(corder))
fingerprinting_df = fingerprinting_df.rename(columns={'router vendor':'Vendor'})

In [ ]:
print("#######################")
print("######  TABLE 2  ######")
print("#######################")
#print(fingerprinting_df.to_latex(index=False))
fingerprinting_df

In [ ]:
# all known response addresses are already directly accessible
response_addresses = (
    scan_df
    .filter(pl.col("response_type") != "Transparent Forwarder")
    .select("ip_response")
    .unique()
    .to_series()
    .to_list()
)
# we might know some resolver addresses through the a-record as well, but they are part of the shielded resolvers since these resolvers are not publically accessible

# mapping of transparent forwarders to shielded resolvers
tfwd_shielded_df = (
    scan_df
    .filter(pl.col("response_type") == "Transparent Forwarder")
    .filter(~pl.col("ip_response").is_in(response_addresses))
)
# ip addresses of only the shielded resolvers themselves
shielded_resolvers = tfwd_shielded_df.select("ip_response").unique()
top5ases = tfwd_shielded_df.group_by("asn_request").agg(pl.len()).sort(by="len",descending=True).head(5).select(pl.col('asn_request')).to_series().to_list()
print("#######################")
print("######  TABLE 4  ######")
print("#######################")
tfwd_shielded_df.group_by("asn_request").agg(pl.len()).sort(by="len",descending=True)\
      .join(tfwd_shielded_df.filter(pl.col('asn_response').is_in(top5ases)).group_by("asn_response","ip_response").agg(pl.len()).group_by("asn_response").agg(pl.len()).sort(by="len",descending=True),left_on='asn_request',right_on='asn_response')\
      .to_pandas().rename(columns={'asn_request':'ASN','len':'# of Transp. Fwds.','len_right':'# of Shielded Resolvers'})#.to_latex(index=False,float_format="%.0f"))

In [ ]:
fnames = glob.glob(f"{data_dir}/any_dnssec_support/udp_A_DNSSEC_verisign.com_*.csv.gz")

dnssec_df = pl.concat(
    [pl.read_csv(file, separator=";", has_header=False, new_columns=["id","ip_request","ip_response","a_record","ts_request","port","dnsid","dns_pkt_size","dns_recs","dns_flags"]).with_columns(
        pl.lit(file.split("_")[-1].split(".")[0]).alias("resolver_type")
    ) for idx, file in enumerate(fnames)]
)
 
dnssec_df = dnssec_df.with_columns(
    pl.when(pl.col("dns_recs").str.contains("RRSIG"))
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("has_dnssec")
)

dnssec_df = dnssec_df.group_by("ip_response","has_dnssec","resolver_type").agg(pl.len())
dnssec_df_filtered_has = dnssec_df.filter(pl.col("has_dnssec"))
dnssec_df_filtered_has_not = dnssec_df.filter(~pl.col("ip_response").is_in(dnssec_df_filtered_has["ip_response"].implode()))
dnssec_df = pl.concat([dnssec_df_filtered_has,dnssec_df_filtered_has_not])
dnssec_df = dnssec_df.group_by("has_dnssec","resolver_type").agg(
    pl.len().alias("dnssec_amount")
)


In [ ]:
fnames = glob.glob(f"{data_dir}/any_dnssec_support/udp_ANY_EDNS0_verisign.com_*.csv.gz")

any_df = pl.concat(
    [pl.read_csv(file, separator=";", has_header=False, new_columns=["id","ip_request","ip_response","a_record","ts_request","port","dnsid","dns_pkt_size","dns_recs","dns_flags"]).with_columns(
        pl.lit(file.split("_")[-1].split(".")[0]).alias("resolver_type")
    ) for idx, file in enumerate(fnames)]
)
any_df = any_df.with_columns(
    pl.when(pl.col("dns_pkt_size") > 100)
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("has_any")
)

any_df = any_df.group_by("ip_response","has_any","resolver_type").agg(pl.len())

any_df_filtered_has = any_df.filter(pl.col("has_any"))
any_df_filtered_has_not = any_df.filter(~pl.col("ip_response").is_in(dnssec_df_filtered_has["ip_response"].implode()))
any_df = pl.concat([any_df_filtered_has,any_df_filtered_has_not])
any_df = any_df.group_by("has_any","resolver_type").agg(
    pl.len().alias("any_amount")
)

In [ ]:
# all known response addresses are already directly accessible
response_addresses2 = (
    scan_df_aug
    .filter(pl.col("response_type") != "Transparent Forwarder")
    .select("ip_response")
    .unique()
    .to_series()
    .to_list()
)
# we might know some resolver addresses through the a-record as well, but they are part of the shielded resolvers since these resolvers are not publically accessible

# mapping of transparent forwarders to shielded resolvers
tfwd_shielded_df2 = (
    scan_df_aug
    .filter(pl.col("response_type") == "Transparent Forwarder")
    .filter(~pl.col("ip_response").is_in(response_addresses2))
)
# ip addresses of only the shielded resolvers themselves
shielded_resolvers2 = tfwd_shielded_df2.select("ip_response").unique()

In [ ]:
total_shielded = shielded_resolvers2.n_unique()
shielded_any = any_df.filter((pl.col('has_any')) & (pl.col('resolver_type')=='shielded')).select(pl.col('any_amount')).item()
shielded_noany = any_df.filter((~pl.col('has_any')) & (pl.col('resolver_type')=='shielded')).select(pl.col('any_amount')).item()

shielded_dnssec = dnssec_df.filter((pl.col('has_dnssec')) & (pl.col('resolver_type')=='shielded')).select(pl.col('dnssec_amount')).item()
shielded_nodnssec = dnssec_df.filter((~pl.col('has_dnssec')) & (pl.col('resolver_type')=='shielded')).select(pl.col('dnssec_amount')).item()

total_unshielded = scan_df_aug.filter(pl.col('response_type')=='Resolver').n_unique()
unshielded_any = any_df.filter((pl.col('has_any')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('any_amount')).item()
unshielded_noany = any_df.filter((~pl.col('has_any')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('any_amount')).item()

unshielded_dnssec = dnssec_df.filter((pl.col('has_dnssec')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('dnssec_amount')).item()
unshielded_nodnssec = dnssec_df.filter((~pl.col('has_dnssec')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('dnssec_amount')).item()

table_tmp = pd.DataFrame({'Query':['DNSSEC']*3+['ANY']*3,
                          'Support':[True,False,'n/a']*2,
                          'Shielded Res. [#]':[shielded_dnssec,
                                               shielded_nodnssec,
                                               total_shielded-shielded_dnssec-shielded_nodnssec,
                                               shielded_any,
                                               shielded_noany,
                                               total_shielded-shielded_any-shielded_noany
                                              ],
                          'Shielded Res. [%]':[round(shielded_dnssec/total_shielded*100,2),
                                               round(shielded_nodnssec/total_shielded*100,2),
                                               round((total_shielded-shielded_dnssec-shielded_nodnssec)/total_shielded*100,2),
                                               round(shielded_any/total_shielded*100,2),
                                               round(shielded_noany/total_shielded*100,2),
                                               round((total_shielded-shielded_any-shielded_noany)/total_shielded*100,2)
                                              ],
                          'Open Res. [#]':[unshielded_dnssec,
                                               unshielded_nodnssec,
                                               total_unshielded-unshielded_dnssec-unshielded_nodnssec,
                                               unshielded_any,
                                               unshielded_noany,
                                               total_unshielded-unshielded_any-unshielded_noany
                                              ],
                          'Open Res. [%]':[round(unshielded_dnssec/total_unshielded*100,2),
                                               round(unshielded_nodnssec/total_unshielded*100,2),
                                               round((total_unshielded-unshielded_dnssec-unshielded_nodnssec)/total_unshielded*100,2),
                                               # adding 0.01 to correct rounding issues
                                               round(unshielded_any/total_unshielded*100,2)+0.01,
                                               round(unshielded_noany/total_unshielded*100,2),
                                               round((total_unshielded-unshielded_any-unshielded_noany)/total_unshielded*100,2)
                                              ]
                         })

In [ ]:
print("#######################")
print("######  TABLE 5  ######")
print("#######################")
#print(table_tmp.to_latex(index=False))
table_tmp

In [ ]:
anycast_ip_addresses = [
    "8.8.8.8",
    "8.8.4.4",
    "1.1.1.1",
    "1.0.0.1",
    "208.67.222.222",
    "9.9.9.9",
    "45.90.28.118",
    "76.76.2.0",
    "156.154.70.1",
    "199.85.126.10",
    "64.6.64.6",
    "4.2.2.1",
    "74.82.42.42",
    "185.228.168.9",
    "195.46.39.39",
    "8.26.56.26",
    "94.140.14.14",
    "45.11.45.11",
    "216.146.35.35",
    "223.5.5.5",
    "119.29.29.29",
    "101.226.4.6",
    "180.76.76.76",
    "114.114.114.114",
    "1.2.4.8",
    "205.171.2.26",
    "149.112.121.10",
    "80.80.80.80",
    "77.88.8.8",
    "193.58.251.251"
]
anycast_ip_addresses = ['8.8.8.8','1.1.1.1', '8.8.4.4', '208.67.222.222','1.0.0.1', '195.46.39.39', '4.2.2.1','77.88.8.8',
       '114.114.114.114', '223.5.5.5','119.29.29.29', '193.58.251.251','1.2.4.8',]
       
corresponding_companies = ['Google','Cloudflare','Google','OpenDNS','Cloudflare',
'SafeDNS','Level3 DNS','Yandex DNS','114DNS','AliDNS','DNSPod','SkyDNS','CNNIC DNS']

In [18]:
anycast_df = scan_df.filter((pl.col('ip_response')!=pl.col('a_record')) & (pl.col('ip_response').is_in(anycast_ip_addresses)) & (pl.col('response_type')=='Transparent Forwarder')).to_pandas()#.org_response.unique()

In [19]:
anycast_df['ip_request24'] = anycast_df.ip_request.apply(lambda ip: '.'.join(ip.split('.')[:-1]))
anycast_df['arecord24'] = anycast_df.a_record.apply(lambda ip: '.'.join(ip.split('.')[:-1]))

In [20]:
anycast_df = pl.from_pandas(anycast_df)

In [ ]:
print("#######################")
print("######  TABLE 6  ######")
print("#######################")
#print(anycast_df.group_by('ip_response').agg(
tmp = anycast_df.group_by('ip_response').agg(
    pl.col('country_request').n_unique().alias('Countries [#]'),
    pl.col('ip_request').n_unique().alias('Tfwd [#]'),
    pl.col('ip_request24').n_unique().alias('Tfwd /24 [#]'),
    pl.col('a_record').n_unique().alias('Anycast Infrastructure [#]'),
    pl.col('arecord24').n_unique().alias('Anycast Infrastructure /24 [#]'),
).sort(['Countries [#]','Tfwd [#]','Tfwd /24 [#]'],descending=True).with_columns(
    pl.Series("Company", corresponding_companies)
)

tmp.select([
    "ip_response",
    "Company",
    "Countries [#]",
    "Tfwd [#]",
    "Tfwd /24 [#]",
    "Anycast Infrastructure [#]",
    "Anycast Infrastructure /24 [#]"
]).to_pandas()

In [ ]:
rate_pub_resolver_multi_pop_files

In [23]:
root_path = rate_pub_resolver_multi_pop_files
rl_resolver_multipop = pd.DataFrame([])
for cc_folder in os.listdir(root_path):
    scan_folders = sorted(os.listdir(os.path.join(root_path, cc_folder)))
    for idx, scan_folder in enumerate(scan_folders):
        scan_path = os.path.join(root_path, cc_folder, scan_folder)
        if not os.path.isdir(scan_path) or 'ipynb_checkpoints' in scan_path:
            continue
        tmp = pd.read_csv(os.path.join(scan_path,'rates.csv'), sep=';', names=['resolver_ip','max_rate', 'end_rate'])
        ts = scan_folder.split('_')[0]
        tmp['date'] = pd.to_datetime(ts)
        tmp['country'] = cc_folder
        rl_resolver_multipop = pd.concat([rl_resolver_multipop,tmp],ignore_index=True)

In [ ]:
formatted_df = rl_resolver_multipop.groupby(['resolver_ip','country'])['max_rate'].mean().unstack(level=-1).merge(pd.read_csv(f'{data_dir}/pub_dns_servers/public_dns_servers.txt',sep=';'), left_on='resolver_ip', right_on='resolver_addr').set_index(['resolver_name', 'resolver_addr']).sort_index()

max_rate_per_resolver = formatted_df.max(axis=1).groupby(level='resolver_name').max()
sorted_resolver_order = max_rate_per_resolver.sort_values(ascending=False).index
formatted_df = formatted_df.reindex(sorted_resolver_order, level='resolver_name')

formatted_df = formatted_df[(formatted_df != 0).any(axis=1)]
formatted_df.index.names = ['Resolver name', 'IPv4-address']
formatted_df.columns = pd.MultiIndex.from_product([['Max. rate on avg.'], formatted_df.columns])
formatted_df = formatted_df.astype(int)
formatted_df = formatted_df[formatted_df.index.get_level_values(0).isin(['Google','Cloudflare','OpenDNS','Verisign','AliDNS','CNNIC SDNS','114DNS','DNSPod Public DNS+','Yandex.DNS'])]
order_mapping = {'Google':1,'Cloudflare':2,'OpenDNS':3,'Verisign':4,'AliDNS':5,'CNNIC SDNS':6,'114DNS':7,'DNSPod':8,'Yandex.DNS':9}
formatted_df = formatted_df.sort_index(level=0,key=lambda name: name.map(order_mapping))
tex_table = formatted_df.to_latex(multirow=True, index=True)
print("#######################")
print("######  TABLE 7  ######")
print("#######################")
#print(tex_table)
formatted_df

In [ ]:
orchestration_df = pl.read_csv(f'{data_dir}/tfwds_to_public_dns_servers/2024-10-22_11-27-12_rm-direct_dm-constant_incr-2000ms_max-rate-3000pps/rates.csv',separator=';',has_header=False,new_columns=['Anycast IP address','Packet rate [pps]','Single count'])\
.select(['Anycast IP address','Packet rate [pps]']).sort(by='Packet rate [pps]',descending=True).filter(pl.col('Anycast IP address')!='9.9.9.9')
orchestration_df

In [ ]:
anycast_company = {"8.8.8.8":'Google',"8.8.4.4":'Google',"1.1.1.1":"Cloudflare","1.0.0.1":"Cloudflare","208.67.222.222":"OpenDNS","208.67.220.220":"OpenDNS"}
orchestrated_fwds = []
for ip in list(orchestration_df['Anycast IP address']):
    tmp = pl.read_csv(f'{data_dir}/tfwds_to_public_dns_servers/2024-10-22_11-27-12_rm-direct_dm-constant_incr-2000ms_max-rate-3000pps/{ip}.csv.gz',separator=';',has_header=False,new_columns=['type','ip','rate'])
    orchestrated_fwds.append(len(tmp.filter(pl.col('type')=='rate-data')))
print(orchestrated_fwds)

In [24]:
custom_order={'Google':1,'Cloudflare':2,'OpenDNS':3}

In [25]:
orchestration_df = orchestration_df.with_columns(
    pl.col('Anycast IP address').replace(anycast_company).alias('Company')
)[['Company','Anycast IP address','Packet rate [pps]']]

In [ ]:
orchestration_df = orchestration_df.insert_column(2,pl.Series("Transp. fwd. [#]", orchestrated_fwds))

In [ ]:
# These values are taken from Table 7.
orchestration_df = pl.from_pandas(orchestration_df.to_pandas().sort_values(by='Company',key=lambda x:x.map(custom_order))).insert_column(4,pl.Series('Resolver Packet rate [pps]',[1744, 1773, 822, 874, 733, 829]))

In [ ]:
# Recursive Resolver Packet rate [pps] / Orchestr. Packet rate [pps]
orchestration_df = orchestration_df.insert_column(5,pl.Series("Needed Infrastructure", ['14x','10x','7x','4x','8x','5x'])).to_pandas()

In [ ]:
print("#######################")
print("######  TABLE 8  ######")
print("#######################")
#print(orchestration_df.to_latex(index=False))
orchestration_df